Now we have to put together the different datasets.

In [1]:
import pandas as pd

In [2]:
files_df = pd.read_pickle("data/clean/files_df.pkl")

We remove from the dataframe those rows whose origin is the website (these rows have `WEBSITE` in all the values) and those that come from the API (these rows have `API` in all the values).

In [3]:
files_df.shape

(543341, 30)

In [4]:
website_ids = files_df[files_df.tierafterorder.isin(['WEBSITE'])].index
api_ids = files_df[files_df.tierafterorder.isin(['API'])].index

In [5]:
files_df = files_df[-files_df.tierafterorder.isin(['WEBSITE', 'API'])]

In [6]:
files_df.shape

(493341, 30)

In [7]:
files_df.tierafterorder.value_counts()

T4     212474
T2      75885
T3      70078
VIP     65361
T1      38079
Name: tierafterorder, dtype: int64

In [8]:
scraped_df = pd.read_pickle('data/clean/scraped.pkl')

In [9]:
scraped_df.shape

(40000, 30)

In [10]:
scraped_df = scraped_df.loc[website_ids]

In [11]:
scraped_df.shape

(40000, 30)

In [12]:
scraped_df.head()

,orderportalid,orderdate_gmt,designer,style,shipper,shiptypeid,userid,isvip,country,region,...,freereturn,issale,productid,brand,ddpsubcategory,storeid,countryoforigin,size,category_1stlevel,platform
id,,,,,,,,,,,,,,,,,,,,,
3416a75f4cea9109507cacd8e2f2aefc,32,2018-01-01 00:18:18.093000+00:00,9314,9320,2,2,32.0,Not VIP,1,1,...,1,No,99761,105,NaN,52,34,108,Shoes,app
642e92efb79421734881b53e1e1b18b6,36,2018-01-01 00:20:41.350000+00:00,44,44,2,2,36.0,Not VIP,5,3,...,1,No,44,36,Articles of a kind normally carried in the poc...,35,1,17,Accessories,NaN
3ef815416f775098fe977004015c6193,374773,2018-01-01 00:34:25.690000+00:00,127166,127196,2,2,255071.0,Not VIP,1,1,...,1,No,148017,99,Footwear with outer soles of rubber or plastics,78,26,383,Shoes,web
ac627ab1ccbdb62ec96e702f07f6425b,70,2018-01-01 00:38:28.650000+00:00,72091,72109,2,2,69.0,VIP,16,4,...,1,Yes,80837,459,"Jerseys, pullovers, cardigans, waistcoats and ...",311,2,48,Girls Clothing,app
f899139df5e1059396431415e770c6dd,70,2018-01-01 00:38:28.650000+00:00,2094,2097,2,2,69.0,VIP,16,4,...,1,No,2153,279,"T-shirts, singlets and other vests",106,2,48,Girls Clothing,app


In [13]:
targets_df = pd.read_pickle('data/clean/targets.pkl')
storeid_df = pd.read_pickle('data/clean/storeids.pkl')

In [14]:
api_df = pd.read_pickle('data/clean/api_df.pickle')

We concat the dataframes that have different ids.

In [15]:
train_df = pd.concat(
    [
        pd.concat([files_df, api_df], sort=True),
        scraped_df
    ],
    sort=True
)

In [16]:
train_df.shape

(543341, 32)

Now we join the files that share an index.

In [17]:
train_df = train_df.drop(columns=['returned', 'storeid']).join(targets_df).join(storeid_df)

In [18]:
train_df.shape

(543341, 32)

In [19]:
train_df.to_pickle('data/clean/train_df_merged.pkl')